# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [31]:
# YOUR CHANGES HERE

import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error

df = pd.read_csv("strawberry-prices.tsv", sep="\t", parse_dates=["month"])

train = df[df["month"] < "2024-01-01"].set_index("month")
test_2024 = df[df["month"].dt.year == 2024].set_index("month")

model = SARIMAX(train["price"], order=(1,1,1), seasonal_order=(1,1,1,12))
results = model.fit(disp=False)

forecast_2024 = results.predict(start="2024-01-01", end="2024-12-01")

mae_2024 = mean_absolute_error(test_2024["price"], forecast_2024)
print("2024 Mean Absolute Error:", mae_2024)

backtest_df = forecast_2024.reset_index()
backtest_df.columns = ["month", "price"]
backtest_df.to_csv("strawberry-backtest.tsv", sep="\t", index=False)


/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/python/3.12.1/lib/python3.12/site-packages/stats

2024 Mean Absolute Error: 0.41713319181643754


/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Submit "strawberry-backtest.tsv" in Gradescope.

In [ ]:
#This code is exuecuted after failing the gradescope test, MAE is 0.4 
#Looking through Piazza, professor note training the model more
#So will try Facebook Prophet


import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error

#in Facebook Prophet, "ds" is the datestamp, "y" is the Target Variable, "yhat" is the predicted value

df = pd.read_csv("strawberry-prices.tsv", sep="\t", parse_dates=["month"])
df = df.rename(columns={"month": "ds", "price": "y"})

train = df[df["ds"] < "2024-01-01"]
test_2024 = df[(df["ds"].dt.year == 2024)]

model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
model.fit(train)

future = model.make_future_dataframe(periods=12, freq='MS')  

forecast = model.predict(future)


forecast_2024 = forecast[forecast["ds"].dt.year == 2024][["ds", "yhat"]]

merged = pd.merge(test_2024, forecast_2024, on="ds")
mae_2024 = mean_absolute_error(merged["y"], merged["yhat"])
print("2024 Prophet MAE:", mae_2024)



15:43:10 - cmdstanpy - INFO - Chain [1] start processing
15:43:10 - cmdstanpy - INFO - Chain [1] done processing


2024 Prophet MAE: 0.2345901387067971


## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [33]:
#this changed based on Facebook Prophet

merged = pd.merge(test_2024, forecast_2024[["ds", "yhat"]], on="ds")

residuals = merged["y"] - merged["yhat"]

residual_mean = residuals.mean()
residual_std = residuals.std()

print("Mean of residuals:", residual_mean)
print("Standard deviation of residuals:", residual_std)

accuracy_df = pd.DataFrame({
    "mean": [residual_mean],
    "std": [residual_std]
})

accuracy_df.to_csv("backtest-accuracy.tsv", sep="\t", index=False)


Mean of residuals: -0.03477350807363696
Standard deviation of residuals: 0.274374327235493


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [34]:
# YOUR CHANGES HERE# 

full_train = df[df["ds"] < "2025-01-01"].set_index("ds")

model_2025 = SARIMAX(full_train["y"], order=(1,1,1), seasonal_order=(1,1,1,12))
results_2025 = model_2025.fit(disp=False)

forecast_2025 = results_2025.predict(start="2025-01-01", end="2025-12-01")

forecast_2025_df = forecast_2025.reset_index()
forecast_2025_df.columns = ["month", "price"]

forecast_2025_df.to_csv("strawberry-forecast.tsv", sep="\t", index=False)



/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/python/3.12.1/lib/python3.12/site-packages/stats

Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [35]:
# YOUR CHANGES HERE

import pandas as pd
from itertools import combinations

forecast_df = pd.read_csv("strawberry-forecast.tsv", sep="\t", parse_dates=["month"])
forecast_df["month_str"] = forecast_df["month"].dt.strftime("%Y-%m-%d")

capital = 1_000_000
results = []

# Loop over all valid (buy, sell) month pairs
for i in range(len(forecast_df)):
    for j in range(i + 1, len(forecast_df)):
        buy_row = forecast_df.iloc[i]
        sell_row = forecast_df.iloc[j]

        buy_price = buy_row["price"]
        sell_price = sell_row["price"]
        months_stored = j - i

        adjusted_sell_price = sell_price * 0.9
        total_cost_per_pint = buy_price + 0.20 + 0.10 * months_stored
        profit_per_pint = adjusted_sell_price - total_cost_per_pint

        if profit_per_pint > 0:
            pints = capital // total_cost_per_pint
            total_profit = pints * profit_per_pint

            results.append({
                "buy_month": buy_row["month_str"],
                "sell_month": sell_row["month_str"],
                "pints_purchased": int(pints),
                "expected_profit": round(total_profit, 2)
            })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="expected_profit", ascending=False)

results_df.to_csv("timings.tsv", sep="\t", index=False)


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [36]:
# YOUR CHANGES HERE

import pandas as pd

timings_df = pd.read_csv("timings.tsv", sep="\t")
best_row = timings_df.iloc[0]

buy_month = best_row["buy_month"]
sell_month = best_row["sell_month"]
pints = best_row["pints_purchased"]
best_profit = best_row["expected_profit"]

accuracy_df = pd.read_csv("backtest-accuracy.tsv", sep="\t")
std_dev = accuracy_df["std"].iloc[0]

forecast_df = pd.read_csv("strawberry-forecast.tsv", sep="\t", parse_dates=["month"])
forecast_df["month_str"] = forecast_df["month"].dt.strftime("%Y-%m-%d")

buy_price = forecast_df[forecast_df["month_str"] == buy_month]["price"].values[0]
sell_price = forecast_df[forecast_df["month_str"] == sell_month]["price"].values[0]

buy_idx = forecast_df[forecast_df["month_str"] == buy_month].index[0]
sell_idx = forecast_df[forecast_df["month_str"] == sell_month].index[0]
months_stored = sell_idx - buy_idx

adjusted_sell_price = (sell_price - std_dev) * 0.9
cost_per_pint = buy_price + 0.20 + 0.10 * months_stored
profit_per_pint = adjusted_sell_price - cost_per_pint
one_std_profit = round(profit_per_pint * pints, 2)

check_df = pd.DataFrame([{
    "best_profit": round(best_profit, 2),
    "one_std_profit": one_std_profit
}])
check_df.to_csv("check.tsv", sep="\t", index=False)




Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [37]:
acknowledgements = [
['1. https://www.geeksforgeeks.org/python/complete-guide-to-sarimax-in-python/', 
 'To learn more about Sarimax forecasting which is a statistical method'],
['2. https://www.geeksforgeeks.org/dsa/python-program-for-stock-buy-sell-to-maximize-profit/',
 'To learn more about how python code for maximizing profit'],
]

columns = ['Source', 'Used For/Reason']

ack_df = pd.DataFrame(acknowledgements, columns=columns)

ack_df


,Source,Used For/Reason
0,1. https://www.geeksforgeeks.org/python/comple...,To learn more about Sarimax forecasting which ...
1,2. https://www.geeksforgeeks.org/dsa/python-pr...,To learn more about how python code for maximi...


In [38]:
ack_df.to_csv("acknowledgements.txt", sep="\t", index=False)

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.